## 5:c - Primal-Dual Int Pt. - v1 (Eliminating $\lambda $)

In [213]:
# Imports
import matplotlib.pyplot as plt
import numpy as np
from numpy import log
import shutil
import sys
import os.path

Given function:
\begin{aligned}
    \min_{x_1, x_2}\quad & \frac{3}{x_1 + x_2} + e^{x_1} + (x_1 - x_2 )^2 \\
    \text{s.t. }\quad & x_1\geq 0 \\
    & x_2\geq 0 \\
    & x_{1}^2 + x_{2}^{2} \leq 2\\ 
    & x_1 - x_2 \leq 1 
\end{aligned}


Barrier Function B(x): t f(x) + $\phi$ (x)

In [214]:
# Defining our function
def my_f(x):    
    val = np.exp(x[0]) + 3/(x[0] + x[1]) + (x[0] - x[1])**2
    return val

In [215]:
# Defining the constraint functions
def my_h1(x):    
    val = -x[0]
    return val

def my_h2(x):    
    val = -x[1]
    return val

def my_h3(x):    
    val = x[0]**2 + x[1]**2 -2
    return val

def my_h4(x):    
    val = x[0] - x[1] -1
    return val

### Defining g Function:

$ g(x,y) = f_0 (x) - \sum_{i=1}^{4} \frac{f_i (x)}{t f_i (y)} $ 

In [216]:
# Defining raw function g s.t. Nabla g = r

def g(x,y,t):
    v = my_f(x) + (-1/t)*(1/my_h1(y)) * my_h1(x) + (-1/t)*(1/my_h2(y)) * my_h2(x) + (-1/t)*(1/my_h3(y)) * my_h3(x) + (-1/t)*(1/my_h4(y)) * my_h4(x)
    return v

$ \nabla g_x (x,x) = r(x) $

In [217]:
# Defining r function
def nabla_gx(x,y,t):
    x = x.flatten()
    y = y.flatten()
    #y = x
    h = 1e-5  # Step size
    # Initialize an array to store the partial derivatives
    grad = np.zeros_like(x)
    
    # Compute the partial derivative for each dimension of x
    for i in range(len(x)):
        # Perturb the i-th dimension by h
        x_plus_h = x.copy()
        x_plus_h[i] += h
        
        # Compute the forward difference
        grad[i] = (g(x_plus_h,y,t) - g(x,y,t)) / h
    return grad

In [218]:
def r(x,t):
    y = x
    v = nabla_gx(x,y,t)
    return v

$ \nabla^2 g_{xx} (x,x) = r' (x) - \sum_{i=1}^m \frac{1}{t f_i(x)^2} \nabla f_i(x) \nabla f_i(x)^T$

In [219]:
def nabla2_gx(x,y,t):
    x = x.flatten()
    y = y.flatten()
    h = 1e-5  # Step size

    identity_matrix = np.eye(len(x))  # Identity matrix
    # Construct the perturbation matrix with h values along the diagonal
    h_matrix = h * identity_matrix

    # w.r.t. x
    # Calculate the forward differences for all components simultaneously
    x_perturbed_values = np.array([nabla_gx(x + h_vec,y,t) for h_vec in h_matrix])

    # Calculate the second derivative approximation
    nabla2_g_x = (x_perturbed_values - nabla_gx(x,y,t)) / h
    reshaped_nabla2_gx  = np.reshape(nabla2_g_x, (2, 2))

    v = reshaped_nabla2_gx  
    return v

### Defining

$\sum_{i=1}^m \frac{1}{t f_i(x)^2} \nabla f_i(x) \nabla f_i(x)^T$

In [220]:
def nabla_h(x):
    x = x.flatten()
    h = 1e-5  # Step size
    # Initialize an array to store the partial derivatives
    grad1 = np.zeros_like(x)
    grad2 = np.zeros_like(x)
    grad3 = np.zeros_like(x)
    grad4 = np.zeros_like(x)
    
    # Compute the partial derivative for each dimension of x
    for i in range(len(x)):
        # Perturb the i-th dimension by h
        x_plus_h = x.copy()
        x_plus_h[i] += h
        
        # Compute the forward difference
        grad1[i] = (my_h1(x_plus_h) - my_h1(x)) / h
        grad2[i] = (my_h2(x_plus_h) - my_h2(x)) / h
        grad3[i] = (my_h3(x_plus_h) - my_h3(x)) / h
        grad4[i] = (my_h4(x_plus_h) - my_h4(x)) / h
        grad1 = np.reshape(grad1, (2,1))
        grad2 = np.reshape(grad2, (2,1))
        grad3 = np.reshape(grad3, (2,1))
        grad4 = np.reshape(grad4, (2,1))
    return grad1, grad2, grad3, grad4

def d_nabla_gx_dy(x,y,t):
    grad1x, grad2x, grad3x, grad4x = nabla_h(x)
    grad1y, grad2y, grad3y, grad4y = nabla_h(y)
    v = (1/t)*(my_h1(y)**2)*np.dot(grad1x,grad1y.T) + (1/t)*(my_h2(y)**2)*np.dot(grad2x,grad2y.T) + (1/t)*(my_h2(y)**2)*np.dot(grad2x,grad2y.T) + (1/t)*(my_h4(y)**2)*np.dot(grad4x,grad4y.T)
    return v

$ r' (x) = \nabla^2 g_{xx} (x,x) + \sum_{i=1}^m \frac{1}{t f_i(x)^2} \nabla f_i(x) \nabla f_i(x)^T$

In [221]:
def nabla_r(x,t):
    y = x
    v = nabla2_gx(x,y,t) + d_nabla_gx_dy(x,y,t)
    return v

In [222]:
# Initialising Parameters
x_start = np.array([0.5,0.5])
mu = 1.2
m = 4
t = 1
eps = 1e-5

# Parameter for Backtracking
alpa = 0.5
beta = 0.3

### Backtracking Line Search

In [223]:
def Backtrack_s(x_start,delta_x,t):
    s = 0.999

    x_new = x_start + s * delta_x
    while ( (my_h1(x_new) > 0) & (my_h2(x_new) > 0) & (my_h3(x_new) > 0) & (my_h4(x_new) > 0) ):
        s = beta * s
        x_new = x_start + s * delta_x

    r_old = r(x_start,t)
    r_norm_old = np.dot(r_old.flatten(),r_old.flatten())**0.5
    r_new = r(x_new,t)
    r_norm_new = np.dot(r_new.flatten(),r_new.flatten())**0.5
    while ( r_norm_new > (1 - alpa * s) * r_norm_old ):
        s = beta * s
        x_new = x_start + s * delta_x
        r_new = r(x_new,t)
        r_norm_new = np.dot(r_new.flatten(),r_new.flatten())**0.5
    
    return s    

### Primal Dual Int Pt. Method- v1 Implementation

In [224]:
iter = 0
while True:
    inv_nabla_r = np.linalg.inv(nabla_r(x_start,t)) 
    delta_x = - np.dot(inv_nabla_r,r(x_start,t).flatten()) 

    s = Backtrack_s(x_start,delta_x,t)  
    x_start = x_start + s * delta_x
    r_dual = r(x_start,t)
    norm_r_dual = np.dot(r_dual.flatten(),r_dual.flatten())**0.5

    if (norm_r_dual < eps):
        break
    t = t*mu
    iter += 1

print("Optimal solution:", x_start)
fopt = my_f(x_start)
print("Optimal function value:", fopt)
print("Number of iterations taken to converge:", iter)

Optimal solution: [0.64672118 1.12371685]
Optimal function value: 3.8312911796211995
Number of iterations taken to converge: 41
